In [1]:
import tensorflow as tf
import torch
import os
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img


2024-11-05 01:05:21.961110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 01:05:21.979567: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 01:05:21.985191: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 01:05:22.000338: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # Replace "0" with the appropriate GPU index


In [6]:
def augment_image(image):
    # Random horizontal flip
    image = tf.image.random_flip_left_right(image)

    # Random rotation
    image = tf.image.rot90(image, k=np.random.randint(0, 4))

    # Random brightness
    image = tf.image.random_brightness(image, 0.2)

    # Random contrast
    image = tf.image.random_contrast(image, 0.7, 1.3)

    # Convert back to numpy
    image = np.array(image)

    # Adding Gaussian noise
    noise = np.random.normal(0, 0.05, image.shape)
    image = np.clip(image + noise, 0, 1)

    return image





In [4]:
# Define paths
input_folder_low = 'newdataset/train/low'
input_folder_high = 'newdataset/train/high'
output_folder_low = 'newdataset/train/low'
output_folder_high = 'newdataset/train/high'

# Create output directories if they don't exist
#os.makedirs(output_folder_low, exist_ok=True)
#os.makedirs(output_folder_high, exist_ok=True)

In [7]:
def load_and_augment_images(input_folder, output_folder, target_count):
    image_files = os.listdir(input_folder)
    count = 0
    existing_images = len(image_files)

    # Loop until we have the desired number of augmented images
    while count < target_count:
        for img_file in tqdm(image_files):
            img_path = os.path.join(input_folder, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Error: Could not load image at path: {img_path}")
                continue  # Skip this image if it couldn't be loaded

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

            # Normalize image to [0, 1] range
            img = img / 255.0

            # Augment the image
            augmented_img = augment_image(img)

            # Convert back to [0, 255] for saving
            augmented_img = (augmented_img * 255).astype(np.uint8)

            # Save augmented image to the output folder
            output_path = os.path.join(output_folder, f"faugr_{count}.jpg")
            cv2.imwrite(output_path, cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))

            count += 1
            if count >= target_count:
                break

# Augment low water level images
load_and_augment_images(input_folder_low, output_folder_low, target_count=1186)

# Augment high water level images
load_and_augment_images(input_folder_high, output_folder_high, target_count=1348)

 80%|████████████████████████████████████████████████████████████████▍                | 648/814 [00:56<00:13, 12.10it/s]

Error: Could not load image at path: newdataset/train/low/desktop.ini


  7%|█████▍                                                                            | 43/652 [00:04<01:10,  8.63it/s]
